In [1]:
import os
import cv2
import shutil
# import deeplabcut
import numpy as np
import pandas as pd
import subprocess
import datetime 
import math
from joblib import Parallel, delayed
import glob

# import argparse
from matplotlib import pyplot as plt
from PIL import Image
# from scipy.linalg import fractional_matrix_power

import plotly.express as px
from IPython.display import HTML

from dash import Dash, dcc, html, Input, Output, no_update, callback
import plotly.graph_objects as go
import base64

In [2]:
def img_get(frame_number, vids):
    path = dict([(2,'/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampled_72_2_5_5_AGC_claheDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4'),
                (3,'/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampledDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4'),
                (1,'/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vids_GCDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4')])
    jpg_as_text = []
    for i in vids:
        cap = cv2.VideoCapture(path[i])
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
        retval, image = cap.read()
        retval, buffer = cv2.imencode('.jpg', image)
        jpg_as_text.append(base64.b64encode(buffer).decode())
    
    return jpg_as_text
    

In [3]:
# img_get(12334, [1,3])[1]

In [4]:
# Extracting File names, titles and date created to extrach info from their DLC processing

project_name = 'DLC'
your_name = 'YR'

os.chdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids')
h5files = []
vid_to_crop = []

today_1 = datetime.date(2023, 11, 14)
# today_1 = datetime.date.today()
today = str(today_1)
# today = "2023-10-29"
chkdate = today_1.strftime("%b%d")

proj_fold = project_name+'-'+your_name+'-'+today
# print(today)
target = os.path.join(os.getcwd(),proj_fold,'videos')
# print(target)
for file in os.listdir(target):
    if file.endswith("filtered.h5"):
        h5files.append(os.path.join(target,file))
for file in os.listdir(target):   
    if file.endswith("labeled.mp4"):
        vid_to_crop.append(os.path.join(target,file))
        
crop_source = []
if len(h5files)==len(vid_to_crop):
    for i in h5files:
        found = ''
        for j in vid_to_crop:
            x = os.path.splitext(i)[0].rstrip('_filtered.h5')
            y = os.path.splitext(j)[0].rstrip('filtered_labeled.mp4')
            if x==y:
                found = j
        k = os.path.basename(i)
        l = 'DLC' + k.split('DLC')[1] + 'DLC' + k.split('DLC')[2].rstrip('_filtered.h5')

        crop_source.append([i,found, l, chkdate])
c_s = np.array(crop_source)

In [5]:
for i in c_s:
    fl_pth, vid_pth, x, date = i

    cap = cv2.VideoCapture(vid_pth)

    tot_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(vid_pth, tot_frame)

/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vids_GCDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869912
/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampledDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869919
/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampled_72_2_5_5_AGC_claheDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869919


In [6]:
# Extracts confidence of each point in each video, grouped under points.

feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

conf = pd.DataFrame()

for i in feature_names:
    df2 = pd.DataFrame()
    mul_cols = []
    for j in c_s:
        fl_pth, vid_pth, x, date = j
        name_check = os.path.basename(vid_pth).split('DLC')[0]
        if name_check.endswith("GC"):
            col_name = "MGC"
        elif name_check.endswith("downsampled"):
            col_name = "NGC"
        else:
            col_name = "AGC"
        tracking_data = pd.read_hdf(fl_pth)
        heading = 'DLC_resnet50_DLC'+ date +'shuffle1_1030000'
        necessary_points = tracking_data[heading][feature_names]
        
        lik = pd.DataFrame(necessary_points[i].likelihood)
        lik = lik.rename(columns={"likelihood" : col_name})
        df2 = pd.concat([df2, lik], axis = 1)
        mul_cols.append((i, col_name))

    df2.columns=pd.MultiIndex.from_tuples(mul_cols)
    conf = pd.concat([conf, df2], axis = 1)



In [7]:
conf_diff = pd.DataFrame()

for i in feature_names:
    df2 = pd.DataFrame()
    cdf = conf[i]
    mul_cols = []
    for j in range(3):
        cur_name = cdf.columns[j]
        prev_name = cdf.columns[j-1]
        col_name = cur_name[0]+prev_name[0]
        diff = pd.DataFrame((cdf[cur_name] - cdf[prev_name]).abs(), columns=[col_name])
        df2 = pd.concat([df2, diff], axis = 1)
        mul_cols.append((i, col_name))
    df2.columns=pd.MultiIndex.from_tuples(mul_cols)
    conf_diff = pd.concat([conf_diff, df2], axis = 1)

In [8]:
# def vid_to_arr(path):
#     cap = cv2.VideoCapture(path)
#     vid_in_arr = np.empty(shape = (1,240,320,3))
#     start_frame = 0
#     total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#     cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
#     frame_number = start_frame
#     for i in range(start_frame, total_frame):
#         ret, frame = cap.read()
#         if not ret:
#             break

#         vid_in_arr = np.append(vid_in_arr, [frame], axis = 0)
#         print(round(frame_number/total_frame*100, 2), end = '\r')
#         frame_number += 1
    
#     video.release()
#     cv2.destroyAllWindows()
#     return vid_in_arr

In [9]:
# vid_to_arr('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampled_72_2_5_5_AGC_claheDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4')

In [10]:
# feat = feature_names[0]
# feat_cv1 = conf_diff[feat]
# feat_cv = feat_cv1.drop(['NM'], axis =1)
# feat_cv.index = feat_cv.index.astype(int)
# # limit = conf[feat][1:100000]
# # limit = conf[feat][conf[feat].gt(0.6).any(axis=1)]
# # limitx = feat_cv[feat_cv.NGC.gt(feat_cv.AGC)]
# # limit = limitx[limitx.gt(0.6).any(axis=1)]
# limit = feat_cv[1:100000]
# fig = px.scatter(limit, title=feat, height=900, hover_data=[limit.index.values])
# fig.update_xaxes(rangeslider_visible=True)
# fig.update_layout(hovermode="x unified")
# fig.add_hline(y=0.9)
# # print(fig)
# fig.show()
# # # # HTML(fig.to_html())

In [33]:

feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

feat = feature_names[0]
feat_cv1 = conf_diff[feat]
# feat_cv1 = conf[feat]

rofr = [0,100000]

def set_range(rfd):
    global rofr
    rofr = rfd
    

@callback(
    Output('confidence_scatter_plot', 'figure'),
    Input('difference_between', 'value'),
    Input('frames', 'value'),
    Input('range_selection', 'value'))
def update_graph(d_bw = 'Manual - AGC', frames = [10000,20000], choice = '0-100000'):
    opt = dict([('Manual - AGC','MA'),
                ('Unmodified - AGC','AN'),
                ('Manual - Unmodified','NM')])
    x = choice.split('-')
    rfd = [int(x[0]), int(x[1])]
    set_range(rfd)
    feat_cv = feat_cv1[rfd[0]:rfd[1]]
    limit = feat_cv[frames[0]:frames[1]]
    fig = go.Figure(data=[
        go.Scatter(
            y=limit[opt[d_bw]],
            mode="markers",
            marker=dict(
                color='red',
            )
        )
    ])

    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(hovermode="x unified")

    # turn off native plotly.js hover effects - make sure to use
    # hoverinfo="none" rather than "skip" which also halts events.
    fig.update_traces(hoverinfo="none", hovertemplate=None)

    fig.update_layout(
        xaxis=dict(title='Confidence'),
        yaxis=dict(title='Frame number'),
        plot_bgcolor='rgba(255,255,255,0.1)'
    )
    return fig

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Dropdown(['0-100000', '100001-200000', '200001-300000', '300001-400000', '400001-500000',
                      '500001-600000', '600001-700000', '700001-800000', '800001-900000', '900001-1000000',
                      '1000001-1100000', '1100001-1200000', '1200001-1300000', '1300001-1400000', '1400001-1500000',
                      '1500001-1600000', '1600001-1700000', '1700001-1800000', f'1800001-{feat_cv1.shape[0]}'], '0-100000', id='range_selection'),
        dcc.RadioItems(
                ['Manual - AGC', 'Unmodified - AGC', 'Manual - Unmodified'],
                'Manual - AGC',
                id='difference_between',
                labelStyle={'display': 'inline-block', 'marginTop': '5px', 'width':'70%'}
            ),
    dcc.Graph(
        id="confidence_scatter_plot", 
        figure=update_graph(), 
        
        style={ 'width': '100%', 'height':'90vh'}, 
        clear_on_unhover=True),
]),
    html.Div([dcc.Tooltip(id="graph-tooltip")]),
    html.Div(dcc.RangeSlider(
        # 0,
        # # # update_graph()[1],
        # 100000,
        rofr[0],
        rofr[1],
        step=10000,
        id='frames',
        value=[rofr[0]+20000,rofr[0]+40000],
    ))
])

@callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("confidence_scatter_plot", "hoverData"),
    Input('difference_between', 'value'),
    Input('frames', 'value'),
    Input('range_selection', 'value'))
def display_hover(hoverData, diff_bw, frm, rsv):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    opt = dict([('Manual - AGC',[1,2]),
                ('Unmodified - AGC',[3,2]),
                ('Manual - Unmodified',[1,3])])
    
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    b64 = img_get(num, opt[diff_bw])
    img_src1 = "data:image/jpeg;base64, " + b64[0]
    img_src2 = "data:image/jpeg;base64, " + b64[1]
    # dfd = 'Frame number - ' + 
    # name = diff_bw + 

    children = [
        html.Div([
            html.Img(src=img_src1, style={"width": "100%", 'object-fit':'contain', 'display':'inline-block'}),
            html.Img(src=img_src2, style={"width": "100%", 'object-fit':'contain', 'display':'inline-block'}),
        ], style={'width': '50vh', 'object-fit':'contain', 'display':'inline-block'}),
        # html.Div([
        #     # html.H2(f"{name}", style={"color": "darkblue", "overflow-wrap": "break-word"})
        # ])
    ]

    return True, bbox, children



if __name__ == "__main__":
    app.run(debug=True)